# Applicazione dataset al NND

In questo notebook viene utilizzata l'architettura NDD al dataset della competition e vengono valutati i risultati.

In [1]:
import os
os.environ['KAGGLE_USERNAME'] = "salvatorenedia" # username from the json file
os.environ['KAGGLE_KEY'] = "8f3e25379978ca9b659620807aee8316" # key from the json file
!kaggle competitions download -c m5-forecasting-accuracy

import zipfile
with zipfile.ZipFile("/content/sales_train_validation.csv.zip", 'r') as zip_ref:
    zip_ref.extractall("/content")


sell_prices.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
calendar.csv: Skipping, found more recently modified local copy (use --force to force download)
sales_train_validation.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime as dt
import random 
from itertools import cycle
from ipywidgets import widgets, interactive
from tqdm.autonotebook import tqdm
import datetime
import seaborn as sns


tqdm.pandas()
pd.set_option('max_columns', 50)
plt.style.use('bmh')
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']
color_cycle = cycle(plt.rcParams['axes.prop_cycle'].by_key()['color'])
df_calendar = pd.read_csv('/content/calendar.csv')
df_calendar = df_calendar.iloc[:1913]
df_sales_train = pd.read_csv('/content/sales_train_validation.csv')
df_toplevel_predicted = pd.read_csv('/content/toplevel_predicted.csv')
df_toplevel_predicted.index = np.arange(1885,1913) 
df_toplevel_predicted.columns = ['sales']

import math
def slip_train_val_test( df_to_split, horizon, p = 0.10):
    
    df_test  = df_to_split[-horizon:]
    df_train = df_to_split.head(len(df_to_split)-horizon)
    
    l = len(df_train)
    size_val = math.floor(p*l)
    df_train = df_to_split.head(l-size_val)
    df_val   = df_train[-size_val:]
    return df_train,df_val,df_test


# Create a data frame for items sales per day with item ids (with Store Id) as columns names  and dates as the index 
df_sales_train['item_store_id'] = df_sales_train.apply(lambda x: x['item_id']+'_'+x['store_id'],axis=1)
DF_Sales = df_sales_train.loc[:,'d_1':'d_1913'].T #transpose
DF_Sales.columns = df_sales_train['item_store_id'].values
DF_Sales=DF_Sales.reset_index(drop = True) 
DF_Sales =  DF_Sales.astype(float)
DF_Sales_train,DF_Sales_val, DF_Sales_test = slip_train_val_test(DF_Sales, horizon=28)

DF_Sales['sales'] = DF_Sales.iloc[:,-30490:].sum(axis=1)
toplevel = DF_Sales['sales'] 
DF_Sales = DF_Sales.iloc[:,:30490]
toplevel_train,toplevel_val, toplevel_test = slip_train_val_test(toplevel, horizon=28)

#convert day and month to dummy
s = pd.Series(df_calendar["weekday"])
x_reg_wday=pd.get_dummies(s)
s = pd.Series(df_calendar["month"])
x_reg_month=pd.get_dummies(s)
x_reg = pd.concat([x_reg_wday, x_reg_month], axis=1)

#print(DF_Sales.loc[:, DF_Sales.columns.str.contains('TX')].shape[1])
x_reg_snap = pd.DataFrame(np.zeros((1913, 30490)), columns=DF_Sales.columns)

x_reg_snap = pd.DataFrame(columns=DF_Sales.columns)
x_reg_snap_CA = pd.concat([df_calendar["snap_CA"] for i in range(12196)], axis=1)
x_reg_snap_TX = pd.concat([df_calendar["snap_TX"] for i in range(9147)], axis=1)
x_reg_snap_WI = pd.concat([df_calendar["snap_WI"] for i in range(9147)], axis=1)        
x_reg_snap    = pd.concat([x_reg_snap_CA, x_reg_snap_TX, x_reg_snap_WI], axis=1)
x_reg_snap.columns=DF_Sales.columns

x_reg = pd.concat([x_reg,toplevel,x_reg_snap],axis=1)
x_reg_train,x_reg_val,x_reg_test = slip_train_val_test(x_reg, horizon=28)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  if __name__ == '__main__':


In [3]:
#conversione dell'input per la rete convoluzionale in sequenze di finestra window_size = 365 circa 1 anno
#e successiva conversione in formato 3d
#a = np.array([i for i in range(100)])
#b = rolling_window(a, 28)
window_size     =  30

def rolling_window(a, window):
    a = np.array(a)
    shape = a.shape[:-1] + (a.shape[-1] - window , window)
    strides = a.strides + (a.strides[-1],)
    return np.lib.stride_tricks.as_strided(a, shape=shape, strides=strides)

sequences = rolling_window(toplevel_train, window_size)
# reshape from [samples, timesteps] into [samples, timesteps, features]
sequences = sequences.reshape((sequences.shape[0], sequences.shape[1],1))
sequences.shape

(1667, 30, 1)

In [4]:
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import tensorflow as tf

seq_length      =  window_size 
series_features =  1
train_x_ext     =  x_reg_train.iloc[window_size:] 
test_y          =  DF_Sales_test
ALPHA           =  0

def mse(y_true, y_pred):
  return K.mean(K.square(y_true - y_pred), axis=-1)

# Switch beteen K.abs and K.square if needed
def constraint(y_true, y_pred):
  return K.mean(K.abs(K.sum(y_pred, axis=1) - K.sum(y_true, axis=1)), axis=-1)

# Write a custom loss function
def custom_loss(y_true, y_pred):
    return (1-ALPHA) * mse(y_true, y_pred) + ALPHA * constraint(y_true, y_pred)


def build_disaggregator(seq_length, series_features, ext_variables, filters, kernel_size, pool_size, units, outputs):     

  input_1  = Input(shape=(ext_variables, ),name= "input_MLP")
  input_2  = Input(shape=(seq_length, series_features),name= "input_CNN")

  out_1 = Dense(units/2, activation="relu")(input_1)
  out_1 = Dense(units/2, activation="relu")(input_1)

  out_2 = Conv1D(filters, kernel_size, activation='relu', padding='same')(input_2)
  out_2 = Conv1D(filters, kernel_size, activation='relu', padding='same')(out_2)
  out_2 = Conv1D(filters, kernel_size, activation='relu', padding='same')(out_2)
  out_2 = Conv1D(filters, kernel_size, activation='relu', padding='same')(out_2)
  out_2 = Conv1D(filters, kernel_size, activation='relu', padding='same')(out_2)
  out_2 = Conv1D(filters, kernel_size, activation='relu', padding='same')(out_2)
  out_2 = Flatten()(out_2)

  out = Concatenate()([out_1, out_2])

  out = Dense(units, activation="relu")(out)
  out = Dense(outputs, activation='linear')(out)

  model = Model([input_1, input_2], out)

  return model



model = build_disaggregator(seq_length=seq_length, series_features=series_features, 
                            ext_variables=train_x_ext.shape[1], 
                            filters=16, kernel_size=32, pool_size=2, 
                            units=512, outputs=test_y.shape[1])
model.summary()
model.compile(optimizer='adam', loss=custom_loss, metrics=[mse, constraint])

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_CNN (InputLayer)          [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, 30, 16)       528         input_CNN[0][0]                  
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 30, 16)       8208        conv1d[0][0]                     
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 30, 16)       8208        conv1d_1[0][0]                   
______________________________________________________________________________________________

In [5]:
train_and_pred = toplevel_train.iloc[-window_size:].append(toplevel_val)
seq_val    = rolling_window(train_and_pred , window_size)
# reshape from [samples, timesteps] into [samples, timesteps, features]
seq_val = seq_val.reshape(seq_val.shape[0], seq_val.shape[1],1)
seq_val.shape

(188, 30, 1)

In [6]:
print("input MLP:",x_reg_train.iloc[window_size:].shape, "input CNN:",sequences.shape)
print(model.input)
#plot_model(model, to_file='model.png')
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30)
model.fit([x_reg_train.iloc[window_size:],sequences], DF_Sales_train[window_size:],validation_data =([x_reg_val,seq_val],DF_Sales_val), epochs=1000, callbacks=[callback])

input MLP: (1667, 30510) input CNN: (1667, 30, 1)
[<tf.Tensor 'input_MLP:0' shape=(None, 30510) dtype=float32>, <tf.Tensor 'input_CNN:0' shape=(None, 30, 1) dtype=float32>]
Epoch 1/1000
53/53 [==============================] - 2s 36ms/step - loss: 88.6045 - mse: 90.0042 - constraint: 9025.6631 - val_loss: 6.9228 - val_mse: 6.9249 - val_constraint: 1533.6400
Epoch 2/1000
53/53 [==============================] - 1s 28ms/step - loss: 7.4382 - mse: 7.4845 - constraint: 1070.4427 - val_loss: 6.7376 - val_mse: 6.7422 - val_constraint: 1874.5273
Epoch 3/1000
53/53 [==============================] - 2s 28ms/step - loss: 7.3375 - mse: 7.3353 - constraint: 931.7643 - val_loss: 7.0910 - val_mse: 7.0956 - val_constraint: 495.7668
Epoch 4/1000
53/53 [==============================] - 2s 29ms/step - loss: 7.3383 - mse: 7.3239 - constraint: 782.7695 - val_loss: 6.3176 - val_mse: 6.3216 - val_constraint: 1242.3774
Epoch 5/1000
53/53 [==============================] - 2s 29ms/step - loss: 7.3538 - mse:

In [11]:
toplevel_train_val = toplevel_train.append(toplevel_val)
prev_train_val = pd.DataFrame(toplevel_train_val.iloc[-window_size:] )
train_and_pred = prev_train_val.append(df_toplevel_predicted)
seq_test = rolling_window(train_and_pred["sales"], window_size)
# reshape from [samples, timesteps] into [samples, timesteps, features]
seq_test = seq_test.reshape(seq_test.shape[0], seq_test.shape[1],1)
#seq_test
x_reg_test["sales"] = df_toplevel_predicted
#x_reg_test

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [12]:
#print("input1:",x_reg_test.shape,"input2", seq_test.shape)
#print(model.input)
output=model.predict([x_reg_test, seq_test])
output.shape
df = pd.DataFrame(data=output, columns=DF_Sales_train.columns)
df

,HOBBIES_1_001_CA_1,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_004_CA_1,HOBBIES_1_005_CA_1,HOBBIES_1_006_CA_1,HOBBIES_1_007_CA_1,HOBBIES_1_008_CA_1,HOBBIES_1_009_CA_1,HOBBIES_1_010_CA_1,HOBBIES_1_011_CA_1,HOBBIES_1_012_CA_1,HOBBIES_1_013_CA_1,HOBBIES_1_014_CA_1,HOBBIES_1_015_CA_1,HOBBIES_1_016_CA_1,HOBBIES_1_017_CA_1,HOBBIES_1_018_CA_1,HOBBIES_1_019_CA_1,HOBBIES_1_020_CA_1,HOBBIES_1_021_CA_1,HOBBIES_1_022_CA_1,HOBBIES_1_023_CA_1,HOBBIES_1_024_CA_1,HOBBIES_1_025_CA_1,...,FOODS_3_803_WI_3,FOODS_3_804_WI_3,FOODS_3_805_WI_3,FOODS_3_806_WI_3,FOODS_3_807_WI_3,FOODS_3_808_WI_3,FOODS_3_809_WI_3,FOODS_3_810_WI_3,FOODS_3_811_WI_3,FOODS_3_812_WI_3,FOODS_3_813_WI_3,FOODS_3_814_WI_3,FOODS_3_815_WI_3,FOODS_3_816_WI_3,FOODS_3_817_WI_3,FOODS_3_818_WI_3,FOODS_3_819_WI_3,FOODS_3_820_WI_3,FOODS_3_821_WI_3,FOODS_3_822_WI_3,FOODS_3_823_WI_3,FOODS_3_824_WI_3,FOODS_3_825_WI_3,FOODS_3_826_WI_3,FOODS_3_827_WI_3
0,0.259887,0.366915,0.090557,2.097054,1.183513,0.964049,0.211897,9.380548,1.368644,0.820083,0.103370,0.654355,0.399634,1.462610,6.392368,6.211771,0.988808,0.079643,7.389410,0.354136,0.635661,0.483167,1.493943,0.433257,0.302373,...,0.688123,9.387127,2.511518,0.066283,3.112057,14.013398,2.967650,0.493431,6.831455,1.080632,0.052727,2.524062,0.072113,5.144251,0.741682,1.558351,2.484209,1.963480,0.297380,2.994068,0.578056,0.269697,0.648544,-0.077943,0.939250
1,0.092418,0.449309,0.109630,2.088497,1.034572,0.835117,0.155500,9.033120,1.296078,0.812983,0.076005,0.683354,0.428242,1.379833,6.602479,6.256462,0.895644,0.116076,6.652493,0.417418,0.655193,0.444562,1.715368,0.458812,0.311810,...,0.640642,10.477132,2.944978,-0.007682,2.868548,16.563028,3.211358,0.616797,5.458036,0.928144,0.046821,2.364154,0.059600,4.247155,0.838024,1.293889,2.909913,2.037157,0.142717,2.619334,0.430356,0.282418,0.415420,-0.117399,0.725037
2,0.118948,0.511685,0.146918,1.905743,1.188082,0.951289,0.053007,8.867282,1.414971,0.788783,0.114979,0.544476,0.441386,1.444025,6.122279,6.561216,0.743497,0.053389,7.242861,0.342601,0.744729,0.457858,1.760992,0.358875,0.290560,...,0.780026,11.659101,2.585111,-0.001631,2.708320,19.865131,3.040869,0.562593,6.637812,0.833269,0.015459,1.802995,0.215785,3.429031,0.718433,1.450976,2.342540,1.843905,0.242451,2.165368,0.370195,0.273618,0.453298,-0.114803,0.832635
3,0.063657,0.531305,0.129621,2.536309,1.599777,0.683172,0.108546,8.952865,1.705109,0.835001,0.079066,0.692121,0.386617,1.331955,8.174087,7.018117,0.971469,0.107930,6.754366,0.359776,0.789146,0.489369,1.909921,0.459573,0.518711,...,0.396270,13.524675,3.372106,-0.027831,2.973611,16.296062,2.801469,0.583211,7.591529,1.136693,0.028873,2.419654,0.096307,4.268941,0.695634,1.194360,2.864329,2.124991,0.098684,2.382996,0.313663,0.385846,0.462635,-0.339975,0.743438
4,0.477904,0.509166,0.193059,2.776067,1.630171,0.826267,0.042376,9.109381,1.266056,0.790186,0.200146,0.486453,0.412682,1.534486,7.859445,6.769051,0.956062,-0.101804,9.323475,0.113259,0.601320,0.477738,1.906659,0.409745,0.439808,...,0.779050,12.862532,2.187752,0.048815,2.287583,18.147936,3.357097,0.573280,10.901632,1.151580,-0.075205,1.762165,0.290414,4.928305,0.471821,1.459498,2.066589,1.835812,0.450867,1.954313,0.292055,0.140112,0.743948,0.011734,1.021336
5,0.108706,0.648353,0.140441,4.050412,1.708852,0.829942,0.059307,7.388636,1.332262,1.327196,0.438326,0.873123,0.558237,1.423874,8.099232,6.963825,1.554146,0.063174,8.313538,0.011496,0.779802,0.650256,2.159798,0.345850,0.979455,...,0.840311,14.131274,3.225971,-0.027508,3.703152,28.276278,3.857924,0.561040,13.508789,1.460457,-0.077633,1.822567,0.197942,12.487599,0.666516,1.275305,1.991459,1.989501,0.309830,3.084474,0.462795,0.518432,1.157690,0.167920,1.025148
6,0.117719,0.575852,0.081079,3.089810,1.653303,1.016116,0.159883,9.945457,1.390459,1.045012,0.371398,0.771478,0.515941,1.545440,7.334729,6.457802,1.128372,0.068936,9.085192,0.068453,0.554600,0.444633,1.591084,0.407625,0.733490,...,0.624964,14.086641,3.540684,0.055909,4.063242,24.453394,4.839222,0.587999,15.882886,1.830949,0.013420,2

In [9]:
DF_Sales_test

,HOBBIES_1_001_CA_1,HOBBIES_1_002_CA_1,HOBBIES_1_003_CA_1,HOBBIES_1_004_CA_1,HOBBIES_1_005_CA_1,HOBBIES_1_006_CA_1,HOBBIES_1_007_CA_1,HOBBIES_1_008_CA_1,HOBBIES_1_009_CA_1,HOBBIES_1_010_CA_1,HOBBIES_1_011_CA_1,HOBBIES_1_012_CA_1,HOBBIES_1_013_CA_1,HOBBIES_1_014_CA_1,HOBBIES_1_015_CA_1,HOBBIES_1_016_CA_1,HOBBIES_1_017_CA_1,HOBBIES_1_018_CA_1,HOBBIES_1_019_CA_1,HOBBIES_1_020_CA_1,HOBBIES_1_021_CA_1,HOBBIES_1_022_CA_1,HOBBIES_1_023_CA_1,HOBBIES_1_024_CA_1,HOBBIES_1_025_CA_1,...,FOODS_3_803_WI_3,FOODS_3_804_WI_3,FOODS_3_805_WI_3,FOODS_3_806_WI_3,FOODS_3_807_WI_3,FOODS_3_808_WI_3,FOODS_3_809_WI_3,FOODS_3_810_WI_3,FOODS_3_811_WI_3,FOODS_3_812_WI_3,FOODS_3_813_WI_3,FOODS_3_814_WI_3,FOODS_3_815_WI_3,FOODS_3_816_WI_3,FOODS_3_817_WI_3,FOODS_3_818_WI_3,FOODS_3_819_WI_3,FOODS_3_820_WI_3,FOODS_3_821_WI_3,FOODS_3_822_WI_3,FOODS_3_823_WI_3,FOODS_3_824_WI_3,FOODS_3_825_WI_3,FOODS_3_826_WI_3,FOODS_3_827_WI_3
1885,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,4.0,2.0,0.0,0.0,5.0,0.0,0.0,1.0,1.0,0.0,0.0,...,5.0,5.0,1.0,0.0,9.0,6.0,3.0,2.0,7.0,1.0,0.0,0.0,0.0,3.0,0.0,5.0,2.0,1.0,0.0,3.0,0.0,0.0,1.0,3.0,7.0
1886,0.0,0.0,0.0,0.0,0.0,1.0,2.0,11.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,2.0,0.0,8.0,0.0,0.0,0.0,3.0,0.0,0.0,...,1.0,4.0,3.0,2.0,6.0,15.0,0.0,0.0,12.0,1.0,0.0,0.0,0.0,4.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
1887,0.0,0.0,0.0,0.0,4.0,3.0,0.0,5.0,0.0,1.0,0.0,0.0,0.0,0.0,4.0,6.0,0.0,0.0,5.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,5.0,10.0,0.0,0.0,8.0,1.0,0.0,1.0,0.0,4.0,1.0,5.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1888,0.0,0.0,0.0,0.0,4.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,1.0,1.0,2.0,4.0,1.0,0.0,21.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,4.0,2.0,2.0,2.0,14.0,0.0,1.0,16.0,1.0,0.0,2.0,0.0,1.0,0.0,3.0,3.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0
1889,0.0,0.0,0.0,3.0,0.0,0.0,1.0,12.0,2.0,1.0,0.0,0.0,0.0,2.0,10.0,32.0,1.0,0.0,5.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,8.0,1.0,0.0,1.0,0.0,0.0,2.0,20.0,2.0,0.0,1.0,0.0,2.0,0.0,1.0,4.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,3.0
1890,0.0,0.0,0.0,1.0,1.0,0.0,0.0,38.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,3.0,1.0,0.0,0.0,3.0,0.0,1.0,...,0.0,13.0,0.0,1.0,14.0,0.0,5.0,1.0,32.0,2.0,0.0,5.0,0.0,5.0,0.0,2.0,3.0,3.0,0.0,2.0,0.0,1.0,1.0,0.0,3.0
1891,1.0,0.0,1.0,2.0,4.0,0.0,0.0,22.0,3.0,3.0,0.0,0.0,2.0,2.0,1.0,6.0,2.0,0.0,24.0,1.0,0.0,0.0,2.0,0.0,2.0,...,0.0,5.0,3.0,2.0,6.0,0.0,4.0,0.0,35.0,5.0,1.0,1.0,0.0,11.0,0.0,0.0,3.0,3.0,1.0,3.0,0.0,1.0,1.0,1.0,1.0
1892,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,3.0,1.0,3.0,0.0,0.0,6.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,5.0,2.0,0.0,5.0,0.0,5.0,0.0,14.0,6.0,0.0,3.0,0.0,8.0,1.0,0.0,1.0,3.0,0.0,3.0,0.0,2.0,2.0,1.0,2.0
1893,4.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,4.0,9.0,3.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,8.0,0.0,2.0,0.0,13.0,1.0,0.0,0.0,0.0,11.0,1.0,0.0,6.0,3.0,0.0,1.0,0.0,0.0,1.0,2.0,3.0
1894,2.0,0.0,0.0,1.0,0.0,3.0,0.0,10.0,0.0,1.0,0.0,1.0,0.0,2.0,2.0,1.0,2.0,0.0,4.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,21.0,4.0,3.0,2.0,0.0,5.0,0.0,19.0,2.0,1.0,2.0,0.0,9.0,0.0,1.0,1.0,2.0,0.0,4.0,0.0,0.0,4.0,1.0,2.0


In [13]:
def rmsse(y_true_h, y_pred_h, y_true):  
    
    N = len(y_true)
    numerator   = K.mean(K.square(y_true_h.values - y_pred_h.values))
    denominator = K.mean(K.square(y_true.values[1:N]-y_true.values[0:N-1]))
    return K.eval(K.sqrt(numerator/tf.cast(denominator, tf.float64)))


#compute mean rmsse bottom level
errors = 0
for i in range(len(DF_Sales_test.columns)):
    #print("series",i,",rmsse:", rmsse(DF_Sales_test.iloc[:,i], df.iloc[:,i], DF_Sales.iloc[0:1885,i]))
    errors+=rmsse(DF_Sales_test.iloc[:,i], df.iloc[:,i], DF_Sales.iloc[0:1885,i])
    
mean_score = errors/30490
mean_score                  

0.9956997568901945